# Carregando Bases

In [55]:
# ==================================================== BASES NECESSÁRIAS ========================================================

# BD_ESTRUTURA_COMERCIAL
import pandas as pd

# Importando bases
caminho_vs = r"C:\Users\alexandreks\MetLife\Planejamento, Relacionamento & Suporte Comercial - Documentos\09. Remuneracao\01. Bases"
bd_producao_vs = pd.read_excel(fr"{caminho_vs}\BD_PRODUCAO_VS.xlsx")

# Importando Estruruta Comercial
caminho_dados_excel = r"C:\Users\alexandreks\MetLife\Planejamento, Relacionamento & Suporte Comercial - Documentos\04. Bases Compartilhadas\01. Dados Excel"
bse_estrutura_comercial = pd.read_excel(fr"{caminho_dados_excel}\BD_ESTRUTURA_COMERCIAL.xlsx")
bse_estrutura_comercial_rep = pd.read_excel(fr"{caminho_dados_excel}\BD_ESTRUTURA_COMERCIAL.xlsx", sheet_name='BD_ESTRUTURA_LA')
bse_estrutura_comercial_sub = pd.read_excel(fr"{caminho_dados_excel}\BD_ESTRUTURA_COMERCIAL.xlsx", sheet_name='BD_ESTRUTURA_LU')

# Arquivo Transição
caminho_transicao = r"C:\Users\alexandreks\MetLife\Planejamento, Relacionamento & Suporte Comercial - Documentos\09. Remuneracao\01. Bases"
bd_transicao_LU = pd.read_excel(fr"{caminho_transicao}\BD_TRANSICAO 03.24.xlsx", sheet_name="LU")
bd_transicao_LA = pd.read_excel(fr"{caminho_transicao}\BD_TRANSICAO 03.24.xlsx", sheet_name="LA")

# PEGANDO DATA CADASTRO DA ESTRUTURA COMERCIAL

In [56]:
import pandas as pd

def merge_dates_between_excel_files_LU(file2_path):
    # Leia o primeiro arquivo Excel
    arquivo1 = bd_transicao_LU

    # Leia o segundo arquivo Excel e selecione a planilha "BD_ESTRUTURA_LU"
    arquivo2 = file2_path

    # Filtro para "OK" na coluna "Verificação"
    arquivo1 = arquivo1[(arquivo1["Verificação"] == "OK") & (arquivo1["Data Caadastro"].dt.year != 2026)]

    # Remova a parte da hora da coluna "Data Caadastro"
    arquivo1['Data Caadastro'] = arquivo1['Data Caadastro'].dt.date

    # Converta a coluna 'DATA CADASTRO' para o tipo datetime
    arquivo2['DATA CADASTRO'] = pd.to_datetime(arquivo2['DATA CADASTRO'])

    # Crie uma nova coluna no arquivo 1 chamada "DATA CADASTRO" e preencha com a primeira correspondência encontrada
    arquivo1['DATA CADASTRO'] = arquivo1['Novo LU'].apply(lambda x: arquivo2.loc[arquivo2['NOME ABREVIADO'] == x, 'DATA CADASTRO'].iloc[0].date() if x in arquivo2['NOME ABREVIADO'].values else None)

    # Se "Novo LU" não for encontrado em "NOME ABREVIADO," pesquise em "LU" e puxe "DATA DESCADASTRO" se encontrado
    for index, row in arquivo1.iterrows():
        if pd.isnull(row['DATA CADASTRO']) and not pd.isnull(row['LU']):
            lu_name = row['LU']
            matching_row = arquivo2[arquivo2['NOME ABREVIADO'] == lu_name]
            if not matching_row.empty:
                arquivo1.at[index, 'DATA CADASTRO'] = matching_row.iloc[0]['DATA DESCADASTRO'].date()

    # Salve o resultado na planilha 'LU'
    lu_df = arquivo1
    return lu_df


def merge_dates_between_excel_files_LA(file2_path):
    # Leia o primeiro arquivo Excel
    arquivo1 = bd_transicao_LA

    # Leia o segundo arquivo Excel e selecione a planilha "BD_ESTRUTURA_LU"
    arquivo2 = file2_path

    # Filtro para "OK" na coluna "Verificação"
    arquivo1 = arquivo1[arquivo1["Verificação"] == "OK"]

    # Converta a coluna 'DATA CADASTRO' para o tipo datetime
    arquivo2['DATA CADASTRO'] = pd.to_datetime(arquivo2['DATA CADASTRO'])

    # Crie uma nova coluna no arquivo 1 chamada "DATA CADASTRO" e preencha com a primeira correspondência encontrada
    arquivo1['DATA CADASTRO'] = arquivo1['Novo LA'].apply(lambda x: arquivo2.loc[arquivo2['NOME ABREVIADO'] == x, 'DATA CADASTRO'].iloc[0].date() if x in arquivo2['NOME ABREVIADO'].values else None)

    la_df = arquivo1
    return la_df

# Chame as funções para preencher as planilhas
df_cadastro_lu = merge_dates_between_excel_files_LU(
    file2_path= bse_estrutura_comercial_sub,

)

df_cadastro_la = merge_dates_between_excel_files_LA(
    file2_path= bse_estrutura_comercial_rep,
)

C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\822344132.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arquivo1['Data Caadastro'] = arquivo1['Data Caadastro'].dt.date
C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\822344132.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arquivo1['DATA CADASTRO'] = arquivo1['Novo LU'].apply(lambda x: arquivo2.loc[arquivo2['NOME ABREVIADO'] == x, 'DATA CADASTRO'].iloc[0].date() if x in arquivo2['NOME ABREVIADO'].values else None)
C:\Users\al

# REALIZADO MENSAL - LU

In [57]:
import pandas as pd

def extrair_nomes():
    # Carregar o primeiro arquivo Excel
    df1 = df_cadastro_lu
    # Extrair os nomes da coluna "Novo LU"
    nomes = df1['Novo LU'].unique().tolist()  # Usar .unique() para obter nomes únicos
    return nomes

def calcular_realizado_mensal(df, nome):
    # Converta a coluna "DATA PAGAMENTO" para o formato de data do Python
    df['DATA PAGAMENTO'] = pd.to_datetime(df['DATA PAGAMENTO'], format='%d/%m/%Y')
    
    # Excluir linhas com Data Pagamento igual a '05/05/2017'
    df = df[df['DATA PAGAMENTO'].dt.year != 2017]

    # Filtrando Campanha VS = Não
    df = df[df['CAMPANHA_VS'] == 'N'] 

    # Filtre as linhas onde "PROPOSTA INVALIDA" é "Não" e "LU" é igual ao nome
    df_filtrado = df[(df['PROPOSTA INVALIDA'] == 'Não') & (df['LU'] == nome)]

    # Agrupe os valores da coluna "PREMIO MENSALISADO" por mês e some
    df_agrupado = df_filtrado.groupby(df_filtrado['DATA PAGAMENTO'].dt.strftime('%Y-%m'))['PREMIO MENSALISADO'].sum().reset_index()

    # Renomeie as colunas para combinação
    df_agrupado = df_agrupado.rename(columns={'DATA PAGAMENTO': 'MES COMPETENCIA', 'PREMIO MENSALISADO': 'REALIZADO MENSAL'})

    return df_agrupado

def main():
    # Carregue o arquivo Excel do segundo conjunto de dados
    df2 = bd_producao_vs

    # Filtrando Campanha VS = Não
    df2 = df2[df2['CAMPANHA_VS'] == 'N'] 

    # Extrair nomes únicos da coluna "Novo LU" do arquivo1
    nomes = extrair_nomes()

    # Inicializar um DataFrame vazio para armazenar os resultados
    resultado_final = pd.DataFrame()

    # Iterar sobre cada nome extraído
    for nome in nomes:
        # Calcular o "REALIZADO MENSAL" para o nome atual
        realizado_mensal = calcular_realizado_mensal(df2, nome)

        # Adicionar o nome ao DataFrame resultado
        realizado_mensal['Novo LU'] = nome

        # Criar uma nova coluna 'Mês' contando a partir do primeiro mês observado para este nome
        realizado_mensal['Mês'] = range(1, len(realizado_mensal) + 1)

        # Concatenar o resultado ao DataFrame final
        resultado_final = pd.concat([resultado_final, realizado_mensal], ignore_index=True)

    resultado_final['Mês REALIZADO MENSAL'] = resultado_final['REALIZADO MENSAL'] * 12

    # Remover entradas duplicadas
    resultado_final = resultado_final.drop_duplicates(subset=['MES COMPETENCIA', 'Novo LU', 'Mês REALIZADO MENSAL'])

    # Salvar o resultado em um novo arquivo Excel contendo todas as colunas desejadas
    colunas_selecionadas = ['Mês', 'Novo LU', 'MES COMPETENCIA', 'Mês REALIZADO MENSAL']
    resultado_final = resultado_final[colunas_selecionadas]
    
    resultado_final.head(5)
    
    return resultado_final

# Exemplo de uso da função:
df_realizado_mensal_lu = main()

C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\3775181174.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATA PAGAMENTO'] = pd.to_datetime(df['DATA PAGAMENTO'], format='%d/%m/%Y')
C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\3775181174.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATA PAGAMENTO'] = pd.to_datetime(df['DATA PAGAMENTO'], format='%d/%m/%Y')
C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\3775181174.py:12: SettingWithCopyWarning: 
A value i

## Realizado Mensal LU ANP

In [58]:
import pandas as pd

def extrair_anp_transicao_novo():
    # Carregar os dois arquivos Excel
    df1 = df_realizado_mensal_lu
    df2 = df_cadastro_lu

    # Realizar uma operação de groupby por "Mês" e "Novo LU" no arquivo 2 e somar as colunas relevantes
    df2_agrupado = df2.groupby(['Mês', 'Novo LU'])[['ANP', 'Transição Novo', 'Razão Social', 'Novo CNPJ', 'Agência', 'Regional']].sum().reset_index()

    # Realizar a fusão (merge) com base nas colunas "Mês" e "Novo LU"
    resultado = pd.merge(df1, df2_agrupado, on=['Mês', 'Novo LU'], how='inner')
    
    return resultado

# Exemplo de uso da função:

df_anp_mensal_lu = extrair_anp_transicao_novo()

# REALIZADO MENSAL - LA

In [59]:
import pandas as pd

def extrair_nomes():
    # Carregar o primeiro arquivo Excel
    df1 = df_cadastro_la
    # Extrair os nomes da coluna "Novo LU"
    nomes = df1['Novo LA'].unique().tolist()  # Usar .unique() para obter nomes únicos
    return nomes

def calcular_realizado_mensal(df, nome):
    # Converta a coluna "DATA PAGAMENTO" para o formato de data do Python
    df['DATA PAGAMENTO'] = pd.to_datetime(df['DATA PAGAMENTO'], format='%d/%m/%Y')
    
    # Excluir linhas com Data Pagamento igual a '05/05/2017
    df = df[df['DATA PAGAMENTO'].dt.year != 2017]

    # Filtrando Campanha VS = Não
    df = df[df['CAMPANHA_VS'] == 'N'] 

    # Filtre as linhas onde "PROPOSTA INVALIDA" é "Não" e "LU" é igual ao nome
    df_filtrado = df[(df['PROPOSTA INVALIDA'] == 'Não') & (df['LA'] == nome)]

    # Agrupe os valores da coluna "PREMIO MENSALISADO" por mês e some
    df_agrupado = df_filtrado.groupby(df_filtrado['DATA PAGAMENTO'].dt.strftime('%Y-%m'))['PREMIO MENSALISADO'].sum().reset_index()

    # Renomeie as colunas para combinação
    df_agrupado = df_agrupado.rename(columns={'DATA PAGAMENTO': 'MES COMPETENCIA', 'PREMIO MENSALISADO': 'REALIZADO MENSAL'})

    return df_agrupado

def main():

    # Carregue o arquivo Excel do segundo conjunto de dados
    df2 = bd_producao_vs

    # Filtrando Campanha VS = Não
    df2 = df2[df2['CAMPANHA_VS'] == 'N'] 

    # Extrair nomes únicos da coluna "Novo LU" do arquivo1
    nomes = extrair_nomes()

    # Inicializar um DataFrame vazio para armazenar os resultados
    resultado_final = pd.DataFrame()

    # Iterar sobre cada nome extraído
    for nome in nomes:
        # Calcular o "REALIZADO MENSAL" para o nome atual
        realizado_mensal = calcular_realizado_mensal(df2, nome)

        # Adicionar o nome ao DataFrame resultado
        realizado_mensal['Novo LA'] = nome

        # Criar uma nova coluna 'Mês' contando a partir do primeiro mês observado para este nome
        realizado_mensal['Mês'] = range(1, len(realizado_mensal) + 1)

        # Concatenar o resultado ao DataFrame final
        resultado_final = pd.concat([resultado_final, realizado_mensal], ignore_index=True)

    resultado_final['Mês REALIZADO MENSAL'] = resultado_final['REALIZADO MENSAL'] * 12

    # Remover entradas duplicadas
    resultado_final = resultado_final.drop_duplicates(subset=['MES COMPETENCIA', 'Novo LA', 'Mês REALIZADO MENSAL'])

    # Salvar o resultado em um novo arquivo Excel contendo todas as colunas desejadas
    colunas_selecionadas = ['Mês', 'Novo LA', 'MES COMPETENCIA', 'Mês REALIZADO MENSAL']
    resultado_final = resultado_final[colunas_selecionadas]
    
    return resultado_final



# Exemplo de uso da função:
df_realizado_mensal_la = main()

C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\2952171663.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATA PAGAMENTO'] = pd.to_datetime(df['DATA PAGAMENTO'], format='%d/%m/%Y')
C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\2952171663.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATA PAGAMENTO'] = pd.to_datetime(df['DATA PAGAMENTO'], format='%d/%m/%Y')
C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\2952171663.py:12: SettingWithCopyWarning: 
A value i

## ANP Mensal LA

In [60]:
import pandas as pd

def extrair_anp_transicao_novo():
    # Carregar os dois arquivos Excel
    df1 = df_realizado_mensal_la
    df2 = df_cadastro_la

    # Realizar uma operação de groupby por "Mês" e "Novo LU" no arquivo 2 e somar as colunas relevantes
    df2_agrupado = df2.groupby(['Mês', 'Novo LA'])[['ANP', 'Transição Novo', 'Razão Social', 'Novo CNPJ', 'AG', 'Região']].sum().reset_index()

    # Realizar a fusão (merge) com base nas colunas "Mês" e "Novo LU"
    resultado = pd.merge(df1, df2_agrupado, on=['Mês', 'Novo LA'], how='inner')
    
    return resultado


df_anp_mensal_la = extrair_anp_transicao_novo()

# ANP ACUMULADO - LA

In [61]:
import pandas as pd

def anp_acumulado():
    # Carregue o arquivo Excel
    df = df_anp_mensal_la

    # Crie uma nova coluna 'ANP ACUMULADO'
    df['ANP ACUMULADO'] = 0

    # Itere sobre cada valor único na coluna 'Novo LA'
    for novo_la in df['Novo LA'].unique():
        filtro = df['Novo LA'] == novo_la
        df.loc[filtro, 'ANP ACUMULADO'] = df[filtro]['ANP'].cumsum()

    return df

df_anp_acumulado_la = anp_acumulado()

# ANP ACUMULADO - LU

In [62]:
import pandas as pd

def anp_acumulado():
    # Carregue o arquivo Excel
    df = df_anp_mensal_lu

    # Crie uma nova coluna 'ANP ACUMULADO'
    df['ANP ACUMULADO'] = 0

    # Itere sobre cada valor único na coluna 'Novo LA'
    for novo_la in df['Novo LU'].unique():
        filtro = df['Novo LU'] == novo_la
        df.loc[filtro, 'ANP ACUMULADO'] = df[filtro]['ANP'].cumsum()

    return df

df_anp_acumulado_lu = anp_acumulado()

# REALIZADO MENSAL ACUMULADO - LA


In [63]:
import pandas as pd

def realizado_acumulado():
    # Carregue o arquivo Excel
    df = df_anp_acumulado_la

    # Renomeie as colunas
    df = df.rename(columns={'Mês REALIZADO MENSAL': 'REALIZADO MENSAL',
                            'Transição Novo': 'VALOR TRANSICAO',
                            'Mês': 'MES',
                            'Novo LA': 'NOVO LA',
                            'Razão Social': 'RAZAO SOCIAL',
                            'Novo CNPJ' : 'NOVO CNPJ',
                            'AG' : 'AGENCIA',
                            'Região' : 'REGIONAL' 
                            })

    # Crie uma nova coluna 'REALIZADO MENSAL ACUMULADO'
    df['REALIZADO MENSAL ACUMULADO'] = 0

    # Itere sobre cada valor único na coluna 'NOVO LA'
    for novo_la in df['NOVO LA'].unique():
        filtro = df['NOVO LA'] == novo_la
        df.loc[filtro, 'REALIZADO MENSAL ACUMULADO'] = df[filtro]['REALIZADO MENSAL'].cumsum()

    # Reordene as colunas
    df = df[['MES', 'AGENCIA', 'REGIONAL' , 'NOVO LA', 'MES COMPETENCIA', 'NOVO CNPJ', 'RAZAO SOCIAL','REALIZADO MENSAL', 'REALIZADO MENSAL ACUMULADO', 'ANP', 'ANP ACUMULADO', 'VALOR TRANSICAO']]

    # Salve o DataFrame de volta no arquivo Excel
    return df

# Exemplo de uso da função

df_realizado_mensal_acumulado_la = realizado_acumulado()

# REALIZADO MENSAL ACUMULADO - LU

In [64]:
import pandas as pd

def realizado_acumulado():
    # Carregue o arquivo Excel
    df = df_anp_acumulado_lu

    # Renomeie as colunas
    df = df.rename(columns={'Mês REALIZADO MENSAL': 'REALIZADO MENSAL',
                            'Transição Novo': 'VALOR TRANSICAO',
                            'Mês': 'MES',
                            'Novo LU': 'NOVO LU',
                            'Razão Social': 'RAZAO SOCIAL',
                            'Novo CNPJ' : 'NOVO CNPJ',
                            'Agência' : 'AGENCIA',
                            'Regional' : 'REGIONAL'
                            })

    # Crie uma nova coluna 'REALIZADO MENSAL ACUMULADO'
    df['REALIZADO MENSAL ACUMULADO'] = 0

    # Itere sobre cada valor único na coluna 'NOVO LU'
    for novo_la in df['NOVO LU'].unique():
        filtro = df['NOVO LU'] == novo_la
        df.loc[filtro, 'REALIZADO MENSAL ACUMULADO'] = df[filtro]['REALIZADO MENSAL'].cumsum()

    # Reordene as colunas
    df = df[['MES', 'AGENCIA', 'REGIONAL' ,'NOVO LU', 'MES COMPETENCIA', 'NOVO CNPJ', 'RAZAO SOCIAL','REALIZADO MENSAL', 'REALIZADO MENSAL ACUMULADO', 'ANP', 'ANP ACUMULADO', 'VALOR TRANSICAO']]

    # Salve o DataFrame de volta no arquivo Excel
    return df

# Exemplo de uso da função
df_realizado_mensal_acumulado_lu = realizado_acumulado()

# ATINGIMENTO MENSAL - LU 

In [65]:
import pandas as pd

def calcular_atingimento_mensal():
    # Carregue o arquivo Excel
    df = df_realizado_mensal_acumulado_lu

    # # Crie uma nova coluna 'ATINGIMENTO MENSAL'
    df['ATINGIMENTO MENSAL'] = df.apply(lambda x: x['REALIZADO MENSAL'] / x['ANP'] 
                                            if x['REALIZADO MENSAL'] != 0 and x['ANP'] != 0
                                            else 0, axis=1)
    
    
    # Salve o DataFrame de volta no arquivo Excel
    return df
# Substitua "seuarquivo.xlsx" pelo nome do seu arquivo Excel de entrada e "saida.xlsx" pelo nome do arquivo de saída.
df_atingimento_mensal_lu = calcular_atingimento_mensal()

# ATINGIMENTO MENSAL - LA 

In [66]:
import pandas as pd

def calcular_atingimento_mensal():
    # Carregue o arquivo Excel
    df = df_realizado_mensal_acumulado_la

    # Crie uma nova coluna 'ATINGIMENTO MENSAL'
    df['ATINGIMENTO MENSAL'] = df.apply(lambda x: x['REALIZADO MENSAL'] / x['ANP'] 
                                            if x['REALIZADO MENSAL'] != 0 and x['ANP'] != 0
                                            else 0, axis=1)

    # Salve o DataFrame de volta no arquivo Excel
    return df

# Substitua "seuarquivo.xlsx" pelo nome do seu arquivo Excel de entrada e "saida.xlsx" pelo nome do arquivo de saída.
df_atingimento_mensal_la = calcular_atingimento_mensal()

# ATINGIMENTO ACUMULADO - LU

In [67]:
import pandas as pd

def calcular_atingimento_mensal():
    # Carregue o arquivo Excel
    df = df_atingimento_mensal_lu

    # Crie uma nova coluna 'ATINGIMENTO MENSAL'
    df['ATINGIMENTO ACUMULADO'] = df.apply(lambda x: x['REALIZADO MENSAL ACUMULADO'] / x['ANP ACUMULADO'] 
                                            if x['REALIZADO MENSAL ACUMULADO'] != 0 and x['ANP ACUMULADO'] != 0
                                            else 0, axis=1)

    # Salve o DataFrame de volta no arquivo Excel
    return df

# Substitua "seuarquivo.xlsx" pelo nome do seu arquivo Excel de entrada e "saida.xlsx" pelo nome do arquivo de saída.
df_atingimento_acumulado_lu = calcular_atingimento_mensal()

# ATINGIMENTO ACUMULADO - LA

In [68]:
import pandas as pd

def calcular_atingimento_mensal():
    # Carregue o arquivo Excel
    df = df_atingimento_mensal_la

    # Crie uma nova coluna 'ATINGIMENTO MENSAL'
    df['ATINGIMENTO ACUMULADO'] = df.apply(lambda x: x['REALIZADO MENSAL ACUMULADO'] / x['ANP ACUMULADO'] 
                                            if x['REALIZADO MENSAL ACUMULADO'] != 0 and x['ANP ACUMULADO'] != 0
                                            else 0, axis=1)

    # Salve o DataFrame de volta no arquivo Excel
    return df

# Substitua "seuarquivo.xlsx" pelo nome do seu arquivo Excel de entrada e "saida.xlsx" pelo nome do arquivo de saída.
df_atingimento_acumulado_la = calcular_atingimento_mensal()

# Pegando MES ATUAL - 1

In [69]:
import pandas as pd
from datetime import datetime

# Obtendo Data atual
data_atual = datetime.now()

# Subtrai um Mês
data_resultante = pd.to_datetime(data_atual) - pd.DateOffset(months=1)

# Subtrai um Ano
data_resultante_anterior = data_resultante - pd.DateOffset(years=1)
data_resultante_anterior = data_resultante_anterior - pd.DateOffset(months=1)

# Mês Formatado
mes_resultante = data_resultante.strftime('%Y-%m')

mes_resultante2 = data_resultante_anterior.strftime('%Y-%m')

mes_arquivo = data_atual.strftime('%m.%Y')

print(mes_resultante, mes_resultante2, mes_arquivo)


2024-02 2023-01 03.2024


# CALCULANDO A PERSISTENCIA - LU

## Fazendo BD PERSISTENCIA VS

In [70]:
import pandas as pd

def extrair_colunas(colunas_desejadas):
    # Lê o arquivo Excel
    df = bd_producao_vs

    # Seleciona apenas as colunas desejadas
    df_selecionado = df[colunas_desejadas]

    # Filtrando Proposta Inválida = Não
    df_selecionado = df_selecionado[df_selecionado['PROPOSTA INVALIDA'] == 'Não']

    # Filtrando Campanha VS = Não
    df_selecionado = df_selecionado[df_selecionado['CAMPANHA_VS'] == 'N']

    # Salva o novo DataFrame em um novo arquivo Excel ou realiza outras operações necessárias
    # df_selecionado.to_excel('BD_PRODUCAO_PERSISTENCIA.xlsx', index=False)
    return df_selecionado
    
# Exemplo de uso
colunas_desejadas = ['LA', 'LU', 'PREMIO MENSALISADO', 'DATA PAGAMENTO', 'DATA CANCELAMENTO APOLICE', 'PROPOSTA INVALIDA', 'CAMPANHA_VS']
df_bd_producao_persistencia  = extrair_colunas(colunas_desejadas)

## Persistencia LU

In [71]:
import pandas as pd
from openpyxl import load_workbook

# Carrega BD_PRODUCAO
bd_producao = df_bd_producao_persistencia

# Filtra PROPOSTA INVALIDA para Não
bd_producao['PROPOSTA INVALIDA'] = 'Não'

# Converte a coluna DATA CANCELAMENTO APOLICE para o formato mês/ano
bd_producao['DATA CANCELAMENTO APOLICE'] = pd.to_datetime(bd_producao['DATA CANCELAMENTO APOLICE'], errors='coerce').dt.strftime('%Y-%m')

# Carrega o Data Apuracao
novo_la = df_atingimento_acumulado_lu

# Função para calcular a taxa de persistência para um NOVO LU específico
def calculo_persistencia(lu):
    # Filtra os valores vazios na coluna DATA PAGAMENTO para o consultor atual
    pagamentos = bd_producao[(bd_producao['LU'] == lu) & bd_producao['DATA PAGAMENTO'].notna()]

    # Converte a coluna 'DATA PAGAMENTO' para o formato ano-mes
    pagamentos['DATA PAGAMENTO'] = pd.to_datetime(pagamentos['DATA PAGAMENTO'], errors='coerce').dt.strftime('%Y-%m')

    # Filtra com base no mês
    mes_pagamento = mes_resultante
    pagamentos = pagamentos[pagamentos['DATA PAGAMENTO'] <= '2023-12']

    cancelamentos = pagamentos[pagamentos['DATA CANCELAMENTO APOLICE'].notna()]

    # Filtra os cancelamentos antes de agosto (ou a data desejada)
    mes_cancelamento = mes_resultante
    cancelamentos = cancelamentos[cancelamentos['DATA CANCELAMENTO APOLICE'] <= '2023-12']

    # Conta o número de apólices canceladas
    apolices_canceladas = len(cancelamentos)

    # Conta o número de apólices ativas sem data de cancelamento
    apolices_ativas = len(pagamentos[pagamentos['DATA CANCELAMENTO APOLICE'].isna()])

    # Conta o número total de apólices
    total_apolices = len(pagamentos['PREMIO MENSALISADO'])

    # Calcula a taxa de persistência em porcentagem
    if apolices_canceladas == 0:
        taxa_persistencia = 100  # Se não houver apólices canceladas, a persistência é 100%
    else:
        taxa_persistencia = apolices_ativas / total_apolices if total_apolices != 0 else 0

    return taxa_persistencia

# Calcula a taxa de persistência para cada NOVO LU e armazena os resultados em uma lista
taxas_persistencia = [calculo_persistencia(lu) for lu in novo_la['NOVO LU']]

# Adiciona a coluna 'PERSISTENCIA' no DataFrame 'novo_lu' com as taxas de persistência calculadas
novo_la['PERSISTENCIA'] = taxas_persistencia

# Salva o resultado em um arquivo Excel com as mesmas colunas de 'novo_lu' e a coluna 'PERSISTENCIA' no final
df_taxa_persistencia_lu = novo_la

print("Processo concluído. Resultados salvos em 'TAXA_PERSISTENCIA_LU.xlsx'")

C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\839551984.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pagamentos['DATA PAGAMENTO'] = pd.to_datetime(pagamentos['DATA PAGAMENTO'], errors='coerce').dt.strftime('%Y-%m')
C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\839551984.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pagamentos['DATA PAGAMENTO'] = pd.to_datetime(pagamentos['DATA PAGAMENTO'], errors='coerce').dt.strftime('%Y-%m')
C:\Users\alexandreks\AppData\Local\Temp\1\

Processo concluído. Resultados salvos em 'TAXA_PERSISTENCIA_LU.xlsx'


C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\839551984.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pagamentos['DATA PAGAMENTO'] = pd.to_datetime(pagamentos['DATA PAGAMENTO'], errors='coerce').dt.strftime('%Y-%m')
C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\839551984.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pagamentos['DATA PAGAMENTO'] = pd.to_datetime(pagamentos['DATA PAGAMENTO'], errors='coerce').dt.strftime('%Y-%m')
C:\Users\alexandreks\AppData\Local\Temp\1\

In [72]:
# ======================================= PARA VER A QTD DE APOLICES CASO PRECISE ======================================



# import pandas as pd
# from openpyxl import load_workbook

# # Carrega BD_PRODUCAO
# bd_producao = pd.read_excel(r"C:\Users\alexandreks\OneDrive - MetLife\Documents\BD_TRANSICAO\BD_PRODUCAO.xlsx")

# # Filtra PROPOSTA INVALIDA para Não
# bd_producao['PROPOSTA INVALIDA'] = 'Não'

# # Converte a coluna DATA CANCELAMENTO APOLICE para o formato mês/ano
# bd_producao['DATA CANCELAMENTO APOLICE'] = pd.to_datetime(bd_producao['DATA CANCELAMENTO APOLICE'], errors='coerce').dt.strftime('%m/%Y')

# # Carrega o Data Apuracao
# novo_lu = pd.read_excel(r"C:\Users\alexandreks\OneDrive - MetLife\Documents\BD_TRANSICAO\ATIGIMENTO_MENSAL_LU.xlsx")

# # Função para calcular a taxa de persistência e trazer as apólices ativas, total de apólices e apólices canceladas
# def calculo_persistencia(lu):
#     # Filtra os valores vazios na coluna DATA PAGAMENTO para o consultor atual
#     pagamentos = bd_producao[(bd_producao['LU'] == lu) & bd_producao['DATA PAGAMENTO'].notna()]

#     cancelamentos = pagamentos[pagamentos['DATA CANCELAMENTO APOLICE'].notna()]

#     # Filtra os cancelamentos antes de agosto (ou a data desejada)
#     cancelamentos = cancelamentos[cancelamentos['DATA CANCELAMENTO APOLICE'].str.split('/').str[0].astype(int) <= 9]

#     # Conta o número de apólices canceladas
#     apolices_canceladas = len(cancelamentos)

#     # Conta o número de apólices ativas sem data de cancelamento
#     apolices_ativas = len(pagamentos[pagamentos['DATA CANCELAMENTO APOLICE'].isna()])

#     # Conta o número total de apólices
#     total_apolices = len(pagamentos['PREMIO MENSALISADO'])

#     # Calcula a taxa de persistência em porcentagem
#     if apolices_canceladas == 0:
#         taxa_persistencia = 100  # Se não houver apólices canceladas, a persistência é 100%
#     else:
#         taxa_persistencia = (apolices_ativas / total_apolices) * 100 if total_apolices != 0 else 0

#     return {
#         'TAXA_PERSISTENCIA': taxa_persistencia,
#         'APOLICES_ATIVAS': apolices_ativas,
#         'TOTAL_APOLICES': total_apolices,
#         'APOLICES_CANCELADAS': apolices_canceladas
#     }

# # Calcula a taxa de persistência e informações das apólices para cada NOVO LU
# resultados = [calculo_persistencia(lu) for lu in novo_lu['NOVO LU']]

# # Converte a lista de resultados em um DataFrame
# resultado_final = pd.DataFrame(resultados)

# # Adicione a coluna 'NOVO LU' de novo_lu ao DataFrame resultado_final
# resultado_final['NOVO LU'] = novo_lu['NOVO LU']

# # Reordene as colunas para colocar 'NOVO LU' na primeira posição
# resultado_final = resultado_final[['NOVO LU', 'TAXA_PERSISTENCIA', 'APOLICES_ATIVAS', 'TOTAL_APOLICES', 'APOLICES_CANCELADAS']]

# # Salva o resultado em um arquivo Excel
# resultado_final.to_excel('taxa_persistencia_PrCadastro.xlsx', index=False)

# print("Processo concluído. Resultados salvos em 'taxa_persistencia_PrCadastro.xlsx'")

# CALCULANDO A PERSISTENCIA - LA

In [73]:
import pandas as pd

# Carrega BD_PRODUCAO
bd_producao = df_bd_producao_persistencia

# Filtra PROPOSTA INVALIDA para Não
bd_producao['PROPOSTA INVALIDA'] = 'Não'

# Converte a coluna DATA CANCELAMENTO APOLICE para o formato mês/ano
bd_producao['DATA CANCELAMENTO APOLICE'] = pd.to_datetime(bd_producao['DATA CANCELAMENTO APOLICE'], errors='coerce').dt.strftime('%Y-%m')

# Carrega o Data Apuracao
novo_la = df_atingimento_acumulado_la

# Função para calcular a taxa de persistência para um NOVO LA específico
def calculo_persistencia(la):
    # Filtra os valores vazios na coluna DATA PAGAMENTO para o consultor atual
    pagamentos = bd_producao[(bd_producao['LA'] == la) & bd_producao['DATA PAGAMENTO'].notna()]

    # Converte a coluna 'DATA PAGAMENTO' para o formato ano-mes
    pagamentos['DATA PAGAMENTO'] = pd.to_datetime(pagamentos['DATA PAGAMENTO'], errors='coerce').dt.strftime('%Y-%m')

    # Filtra com base no mês
    mes_pagamento = mes_resultante
    pagamentos = pagamentos[pagamentos['DATA PAGAMENTO'] <= '2023-12']

    cancelamentos = pagamentos[pagamentos['DATA CANCELAMENTO APOLICE'].notna()]

    # Filtra os cancelamentos antes de agosto (ou a data desejada)
    mes_cancelamento = mes_resultante
    cancelamentos = cancelamentos[cancelamentos['DATA CANCELAMENTO APOLICE'] <= '2023-12']

    # Conta o número de apólices canceladas
    apolices_canceladas = len(cancelamentos)

    # Conta o número de apólices ativas sem data de cancelamento
    apolices_ativas = len(pagamentos[pagamentos['DATA CANCELAMENTO APOLICE'].isna()])

    # Conta o número total de apólices
    total_apolices = len(pagamentos['PREMIO MENSALISADO'])

    # Calcula a taxa de persistência em porcentagem
    if apolices_canceladas == 0:
        taxa_persistencia = 100  # Se não houver apólices canceladas, a persistência é 100%
    else:
        taxa_persistencia = apolices_ativas / total_apolices if total_apolices != 0 else 0

    return taxa_persistencia

# Calcula a taxa de persistência para cada NOVO LA e armazena os resultados em uma lista
taxas_persistencia = [calculo_persistencia(la) for la in novo_la['NOVO LA']]

# Adiciona a coluna 'PERSISTENCIA' no DataFrame 'novo_la' com as taxas de persistência calculadas
novo_la['PERSISTENCIA'] = taxas_persistencia

# Salva o resultado em um arquivo Excel com as mesmas colunas de 'novo_la' e a coluna 'PERSISTENCIA' no final
df_taxa_persistencia_la = novo_la

print("Processo concluído. Resultados salvos em 'TAXA_PERSISTENCIA_LA.xlsx'")

C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\3575543426.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pagamentos['DATA PAGAMENTO'] = pd.to_datetime(pagamentos['DATA PAGAMENTO'], errors='coerce').dt.strftime('%Y-%m')
C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\3575543426.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pagamentos['DATA PAGAMENTO'] = pd.to_datetime(pagamentos['DATA PAGAMENTO'], errors='coerce').dt.strftime('%Y-%m')
C:\Users\alexandreks\AppData\Local\Temp\

Processo concluído. Resultados salvos em 'TAXA_PERSISTENCIA_LA.xlsx'


C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\3575543426.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pagamentos['DATA PAGAMENTO'] = pd.to_datetime(pagamentos['DATA PAGAMENTO'], errors='coerce').dt.strftime('%Y-%m')
C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\3575543426.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pagamentos['DATA PAGAMENTO'] = pd.to_datetime(pagamentos['DATA PAGAMENTO'], errors='coerce').dt.strftime('%Y-%m')
C:\Users\alexandreks\AppData\Local\Temp\

# ELEGIVEL MENSAL - LU

In [74]:
import pandas as pd

def calcular_elegivel_mensal():
    # Carregue o arquivo Excel
    df = df_taxa_persistencia_lu

    # Transforme a coluna 'ATINGIMENTO MENSAL' em porcentagem
    df['ATINGIMENTO MENSAL'] = df['ATINGIMENTO MENSAL'] * 100

    # Crie a coluna 'ELEGIVEL MENSAL' com base nas condições especificadas
    df['ELEGIVEL MENSAL'] = (df['ATINGIMENTO MENSAL'] >= 50) & (df['PERSISTENCIA'] >= 0.9)
    df['ELEGIVEL MENSAL'] = df['ELEGIVEL MENSAL'].apply(lambda x: 'S' if x else 'N')

    df['ATINGIMENTO MENSAL'] = df['ATINGIMENTO MENSAL'] / 100

    # Salve o DataFrame com a nova coluna em um novo arquivo Excel
    return df

# Exemplo de uso da função
df_elegivel_mensal_lu = calcular_elegivel_mensal()

# ELEGIVEL MENSAL - LA

In [75]:
import pandas as pd

def calcular_elegivel_mensal():
    # Carregue o arquivo Excel
    df = df_taxa_persistencia_la

    # Transforme a coluna 'ATINGIMENTO MENSAL' em porcentagem
    df['ATINGIMENTO MENSAL'] = df['ATINGIMENTO MENSAL'] * 100

    # Crie a coluna 'ELEGIVEL MENSAL' com base nas condições especificadas
    df['ELEGIVEL MENSAL'] = (df['ATINGIMENTO MENSAL'] >= 50) & (df['PERSISTENCIA'] >= 0.9)
    df['ELEGIVEL MENSAL'] = df['ELEGIVEL MENSAL'].apply(lambda x: 'S' if x else 'N')

    df['ATINGIMENTO MENSAL'] = df['ATINGIMENTO MENSAL'] / 100

    # Salve o DataFrame com a nova coluna em um novo arquivo Excel
    return df
    
# Exemplo de uso da função
df_elegivel_mensal_la  = calcular_elegivel_mensal()

# ELEGIVEL ACUMULADO - LU

In [76]:
import pandas as pd

def calcular_elegivel_mensal():
    # Carregue o arquivo Excel
    df = df_elegivel_mensal_lu

    # Transforme a coluna 'ATINGIMENTO MENSAL' em porcentagem
    df['ATINGIMENTO ACUMULADO'] = df['ATINGIMENTO ACUMULADO'] * 100

    # Crie a coluna 'ELEGIVEL MENSAL' com base nas condições especificadas
    df['ELEGIVEL ACUMULADO'] = (df['ATINGIMENTO ACUMULADO'] >= 50) & (df['PERSISTENCIA'] >= 0.9)
    df['ELEGIVEL ACUMULADO'] = df['ELEGIVEL ACUMULADO'].apply(lambda x: 'S' if x else 'N')

    df['ATINGIMENTO ACUMULADO'] = df['ATINGIMENTO ACUMULADO'] / 100

    # Salve o DataFrame com a nova coluna em um novo arquivo Excel
    return df

# Exemplo de uso da função
df_elegivel_acumulado_lu = calcular_elegivel_mensal()

# ELEGIVEL ACUMULADO - LA

In [77]:
import pandas as pd

def calcular_elegivel_mensal():
    # Carregue o arquivo Excel
    df = df_elegivel_mensal_la

    # Transforme a coluna 'ATINGIMENTO MENSAL' em porcentagem
    df['ATINGIMENTO ACUMULADO'] = df['ATINGIMENTO ACUMULADO'] * 100

    # Crie a coluna 'ELEGIVEL MENSAL' com base nas condições especificadas
    df['ELEGIVEL ACUMULADO'] = (df['ATINGIMENTO ACUMULADO'] >= 50) & (df['PERSISTENCIA'] >= 0.9)
    df['ELEGIVEL ACUMULADO'] = df['ELEGIVEL ACUMULADO'].apply(lambda x: 'S' if x else 'N')

    df['ATINGIMENTO ACUMULADO'] = df['ATINGIMENTO ACUMULADO'] / 100

    # Salve o DataFrame com a nova coluna em um novo arquivo Excel
    return df

# Exemplo de uso da função
df_elegivel_acumulado_la = calcular_elegivel_mensal()

# VALOR RECEBER - LU

In [78]:
import pandas as pd

def calcular_valor_receber():
    # Carregue o arquivo Excel
    df = df_elegivel_acumulado_lu

    # Renomeie a coluna 'NOVO LU' para 'NOME'
    df = df.rename(columns={'NOVO LU': 'NOME'}) 

    # Filtra as linhas onde 'ELEGIVEL MENSAL' é igual a 'S'
    df_filtrado = df[(df['ELEGIVEL MENSAL'] == 'S') | (df['ELEGIVEL ACUMULADO'] == 'S')]

    # Cria a coluna 'MAX_ATINGIMENTO' com o valor máximo entre 'ATINGIMENTO MENSAL' e 'ATINGIMENTO ACUMULADO'
    df_filtrado['MAX_ATINGIMENTO'] = df_filtrado[['ATINGIMENTO MENSAL', 'ATINGIMENTO ACUMULADO']].max(axis=1)

    # Calcula os valores da coluna 'VALOR RECEBER' com base nas condições
    df_filtrado['VALOR RECEBER'] = df_filtrado.apply(lambda row: row['VALOR TRANSICAO'] if row['MAX_ATINGIMENTO'] >= 1
                                                     else row['VALOR TRANSICAO'] * (row['MAX_ATINGIMENTO']), axis=1)

    # Adicione a coluna 'MAX_ATINGIMENTO' no final do DataFrame original
    df['MAX_ATINGIMENTO'] = df_filtrado['MAX_ATINGIMENTO']

    # Adicione a coluna 'VALOR RECEBER' no final do DataFrame original
    df['VALOR RECEBER'] = df_filtrado['VALOR RECEBER']

    # Salve o DataFrame com a nova coluna em um novo arquivo Excel
    return df

# Exemplo de uso da função
df_valor_receber_lu = calcular_valor_receber()

C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\3988237712.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['MAX_ATINGIMENTO'] = df_filtrado[['ATINGIMENTO MENSAL', 'ATINGIMENTO ACUMULADO']].max(axis=1)
C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\3988237712.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['VALOR RECEBER'] = df_filtrado.apply(lambda row: row['VALOR TRANSICAO'] if row['MAX_ATINGIMENTO'] >= 1


# VALOR RECEBER - LA

In [79]:
import pandas as pd

def calcular_valor_receber():
    # Carregue o arquivo Excel
    df = df_elegivel_acumulado_la

    # Renomeie a coluna 'NOVO LU' para 'NOME'
    df = df.rename(columns={'NOVO LA': 'NOME'}) 

    # Filtra as linhas onde 'ELEGIVEL MENSAL' é igual a 'S'
    df_filtrado = df[(df['ELEGIVEL MENSAL'] == 'S') | (df['ELEGIVEL ACUMULADO'] == 'S')]

    # Cria a coluna 'MAX_ATINGIMENTO' com o valor máximo entre 'ATINGIMENTO MENSAL' e 'ATINGIMENTO ACUMULADO'
    df_filtrado['MAX_ATINGIMENTO'] = df_filtrado[['ATINGIMENTO MENSAL', 'ATINGIMENTO ACUMULADO']].max(axis=1)

    # Calcula os valores da coluna 'VALOR RECEBER' com base nas condições
    df_filtrado['VALOR RECEBER'] = df_filtrado.apply(lambda row: row['VALOR TRANSICAO'] if row['MAX_ATINGIMENTO'] >= 1
                                                     else row['VALOR TRANSICAO'] * (row['MAX_ATINGIMENTO']), axis=1)

    # Adicione a coluna 'MAX_ATINGIMENTO' no final do DataFrame original
    df['MAX_ATINGIMENTO'] = df_filtrado['MAX_ATINGIMENTO']

    # Adicione a coluna 'VALOR RECEBER' no final do DataFrame original
    df['VALOR RECEBER'] = df_filtrado['VALOR RECEBER']

    # Salve o DataFrame com a nova coluna em um novo arquivo Excel
    return df

# Exemplo de uso da função
df_valor_receber_lu = calcular_valor_receber()

C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\965615240.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['MAX_ATINGIMENTO'] = df_filtrado[['ATINGIMENTO MENSAL', 'ATINGIMENTO ACUMULADO']].max(axis=1)
C:\Users\alexandreks\AppData\Local\Temp\1\ipykernel_23916\965615240.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['VALOR RECEBER'] = df_filtrado.apply(lambda row: row['VALOR TRANSICAO'] if row['MAX_ATINGIMENTO'] >= 1


# JUNTANDO LA|LU - ARQUIVO FINAL

In [80]:
import pandas as pd

def criar_arquivo_excel_unificado():
    # Leitura dos arquivos Excel
    df1 = df_valor_receber_lu
    df2 = df_valor_receber_lu

    # Criar um novo arquivo Excel
    with pd.ExcelWriter(f'BONUS TRANSICAO {mes_arquivo}.xlsx') as writer:
        df1.to_excel(writer, sheet_name='LA', index=False)
        df2.to_excel(writer, sheet_name='LU', index=False)

    print("Arquivo BONUS_TRANSICAO.xlsx criado com sucesso.")


arquivo = criar_arquivo_excel_unificado()

Arquivo BONUS_TRANSICAO.xlsx criado com sucesso.
